In [1]:
import numpy as np
import random as rm
import pandas as pd
from random import seed
from random import random
import matplotlib.pyplot as plt
from pylab import *
from scipy.optimize import curve_fit
from scipy.stats import bernoulli, binom
from scipy import integrate

In [2]:
# build the target distribution
mu1, sigma1 = 0, 0.5
mu2, sigma2 = 17.5, 1
# N = 500

# x1 = np.random.normal(mu1, sigma1, N)
# x2 = np.random.normal(mu2, sigma2, N)

# f_pi = (0.4*x1 + 0.6*x2)

## Method 1

In [3]:
# calculate constant c
f_pi = lambda x: 0.4*(1/np.sqrt(2*math.pi*sigma1))*np.exp((-1/2)*((x-mu1)**2/sigma1)) + 0.6*(1/np.sqrt(2*math.pi*sigma2))*np.exp((-1/2)*((x-mu2)**2/sigma2))
pdf_eta = lambda x: np.power(0.4*(1/np.sqrt(2*math.pi*sigma1))*np.exp((-1/2)*((x-mu1)**2/sigma1)) + 0.6*(1/np.sqrt(2*math.pi*sigma2))*np.exp((-1/2)*((x-mu2)**2/sigma2)),0.75)
c = integrate.quad(pdf_eta, -inf, inf)
c = c[0]

In [4]:
# define eta
def f_eta(x):
    return(pdf_eta(x)/c)

#pdf_eta = lambda x: np.power(0.4*(1/np.sqrt(2*math.pi*sigma1))*np.exp((-1/2)*((x-mu1)**2/sigma1)) + 0.6*(1/np.sqrt(2*math.pi*sigma2))*np.exp((-1/2)*((x-mu2)**2/sigma2)),0.75)


In [5]:
# define g
def g(x):
    return(f_pi(x)/f_eta(x))

In [6]:
# P
def p_kernel(y0,sigma,n):
    count = 0
    Yn = y0
    estimator = [y0]
    while count<n:
        y = np.random.normal(Yn,sigma)
        u = np.random.uniform(0,1)
        alpha = min((pdf_eta(y)/pdf_eta(y0)),1)
        if u <= alpha:
            estimator.append(y)
            Yn = y
            count +=1
    return estimator

In [57]:
# K
def k_kernel(x0,sigma):
    count = 0
    Xn = x0
    estimator = [x0]
    while count<1:
        y = np.random.normal(Xn,sigma)
        u = np.random.uniform(0,1)
        alpha = min((pdf_eta(y)/pdf_eta(x0)),1)
        if u <= alpha:
            estimator.append(y)
            Xn = y
            count +=1
    return estimator[-1]

In [8]:
# define S_y
def S_y(y):
    sum = 0
    for i in y:
        sum += g(i)
    avg = sum/len(y)
    return avg

In [9]:
# def prob_list
def prob_list(y):
    prob_list = []
    sum = 0
    for i in y:
        sum += g(i)
    for i in y:
        prob_list.append(g(i)/sum)
    return prob_list

# Prob(y_i) = prob_list[i]

In [10]:
# define phi
def phi_x(x,y):
    result = []
    for i in x:
        if i in y:
            count = y.count(i)
            index = y.index(i)
            result = result.append(prob_list(y)[index]*count)
    result = result/(S_y(y)*len(y))
    return result

In [61]:
# generate bernoulli list
def bernoulli_list(epsilon, n):
    b_list = bernoulli(epsilon).rvs(n)
    return b_list

# generate x
def x_sample(x0, epsilon, y, n, sigma):
    X_list = [x0]
    for i in bernoulli_list(epsilon, n):
        if i == 0:
            X_list.append(k_kernel(x0, sigma))
        if i == 1:
            X_list.append(np.random.choice(y, p = prob_list(y)))
        x0 = X_list[-1]
    return X_list

In [96]:
# expectation
def E_x(X_list):
    sum = 0
    for i in X_list:
        sum += i
    mean = sum/len(X_list)
    return mean

In [27]:
# define epsilon
epsilon1 = 0.05
epsilon2 = 0.25
epsilon3 = 0.5
epsilon4 = 0.75
epsilon5 = 0.95

In [104]:
# simulate Y
n = 20
y = p_kernel(5,1,n)

# simulate single x
x = k_kernel(5,1)

# simulate x
# x_sample(3, epsilon2, y, n, 2)

In [128]:
x_list = x_sample(10.5, epsilon2, y, n, 1)
print(E_x(x_list))

2.061616680784546
